# Libraries

In [1]:
import pandas as pd
import numpy as np

# Loading results

In [ ]:
result_rafael = pd.read_csv('results_rafael.csv')
result_10_rafael = pd.read_csv('results_10_rafael.csv')

result_luiz = pd.read_csv('results_luiz.csv')
result_10_luiz = pd.read_csv('results_10_luiz.csv')

# result_rafael = pd.concat([result_rafael, result_10_rafael])
# result_luiz = pd.concat([result_luiz, result_10_luiz])

result_10_rafael = result_rafael[result_rafael['Qnt'] != 'DySyn-TS']
result_10_luiz = result_luiz[result_luiz['Qnt'] != 'DySyn-TS']

# Ensure the 'MAE' column is numeric
result_rafael['MAE'] = pd.to_numeric(result_rafael['MAE'], errors='coerce')
result_luiz['MAE'] = pd.to_numeric(result_luiz['MAE'], errors='coerce')

grouped_rafael = result_rafael.groupby(['Qnt', 'MFte', 'MFtr']).agg({'MAE': 'mean'}).reset_index()
grouped_luiz = result_luiz.groupby(['Qnt', 'MFte', 'MFtr']).agg({'MAE': 'mean'}).reset_index()

grouped = pd.concat([grouped_rafael, grouped_luiz])

C:\Users\joaop\AppData\Local\Temp\ipykernel_11116\1831204099.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  result_10_luiz = pd.read_csv('results_10_luiz.csv')


# Plots

# Boxplot general

In [73]:
import plotly.express as px

def plot_sorted_boxplot(df):
    # Calculate the median AE for each Qnt
    median_ae = df.groupby('Qnt')['AE'].median().reset_index()

    # Sort the Qnt by the median AE
    sorted_qnt = median_ae.sort_values(by='AE')['Qnt']

    # Reorder the grouped DataFrame based on the sorted Qnt
    df['Qnt'] = pd.Categorical(df['Qnt'], categories=sorted_qnt, ordered=True)
    df = df.sort_values('Qnt')

    # Plot the sorted boxplot
    fig = px.box(df, x='Qnt', y='AE', color='Qnt', title='Boxplot of AE for each Qnt')
    fig.update_layout(xaxis_title='Qnt', yaxis_title='AE', showlegend=True)
    fig.show()

# Example usage
plot_sorted_boxplot(grouped)

C:\Users\joaop\AppData\Local\Temp\ipykernel_11116\2587209695.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

import plotly.express as px

def plot_sorted_boxplot_with_dropdown(df):
    # Create a dropdown widget for MFtr
    mftr_dropdown = widgets.Dropdown(
        options=df['MFtr'].unique(),
        description='MFtr:'
    )

    # Function to update the plot based on the selected MFtr
    def update_plot(change):
        selected_mftr = mftr_dropdown.value
        filtered_df = df[df['MFtr'] == selected_mftr]

        # Calculate the median AE for each Qnt
        median_ae = filtered_df.groupby('Qnt')['AE'].median().reset_index()

        # Sort the Qnt by the median AE
        sorted_qnt = median_ae.sort_values(by='AE')['Qnt']

        # Reorder the grouped DataFrame based on the sorted Qnt
        filtered_df['Qnt'] = pd.Categorical(filtered_df['Qnt'], categories=sorted_qnt, ordered=True)
        filtered_df = filtered_df.sort_values('Qnt')

        # Plot the sorted boxplot
        fig = px.box(filtered_df, x='Qnt', y='AE', color='Qnt', title=f'Boxplot of AE for each Qnt (MFtr = {selected_mftr})')
        fig.update_layout(xaxis_title='Qnt', yaxis_title='AE', showlegend=True)
        
        clear_output(wait=True)
        display(mftr_dropdown)
        display(fig)

    # Attach the update_plot function to the dropdown widget
    mftr_dropdown.observe(update_plot, names='value')

    # Display the dropdown and the initial plot
    display(mftr_dropdown)
    update_plot(None)

# Example usage
plot_sorted_boxplot_with_dropdown(grouped)

Dropdown(description='MFtr:', index=9, options=(np.float64(0.15), np.float64(0.85), np.float64(0.95), np.float…

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px

def plot_boxplot_acccsyn(df):
    # Filter the dataframe for ACCSyn-TS
    df_acccsyn = df[df['Qnt'] == 'ACCSyn-TS']

    # Calculate the difference
    df_acccsyn['Difference'] = df_acccsyn['AE'] - df_acccsyn['Value.dist']

    # Create a dropdown widget for MFtr
    mftr_dropdown = widgets.Dropdown(
        options=df_acccsyn['MFtr'].unique(),
        description='MFtr:'
    )

    # Function to update the plot based on the selected MFtr
    def update_plot(change):
        selected_mftr = mftr_dropdown.value
        filtered_df = df_acccsyn[df_acccsyn['MFtr'] == selected_mftr]

        # Plot the boxplot
        fig = px.box(filtered_df, x='MFte', y='Difference', title=f'Boxplot of Difference for MFtr = {selected_mftr}')
        fig.update_layout(xaxis_title='MFte', yaxis_title='Difference', showlegend=False)
        
        clear_output(wait=True)
        display(mftr_dropdown)
        display(fig)

    # Attach the update_plot function to the dropdown widget
    mftr_dropdown.observe(update_plot, names='value')

    # Display the dropdown and the initial plot
    display(mftr_dropdown)
    update_plot(None)

# Example usage
plot_boxplot_acccsyn(result_10_rafael)

Dropdown(description='MFtr:', index=7, options=(np.float64(0.55), np.float64(0.65), np.float64(0.15), np.float…

## Line Plot

In [3]:
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

def line_plot(grouped):
    mftr_dropdown = widgets.Dropdown(
        options=grouped['MFtr'].unique(),
        description='MFtr:'
    )

    mftr_dropdown.value = grouped['MFtr'].unique()[0]

    def update_plot(change):
        selected_mftr = mftr_dropdown.value
        filtered_data = grouped[grouped['MFtr'] == selected_mftr]

        fig = go.Figure()
        for qnt_val in filtered_data['Qnt'].unique():
            qnt_data = filtered_data[filtered_data['Qnt'] == qnt_val]
            fig.add_trace(go.Scatter(
                x=qnt_data['MFte'], 
                y=qnt_data['MAE'], 
                mode='lines+markers', 
                name=qnt_val,
                visible='legendonly'  # Set initial visibility to legendonly
            ))

        fig.update_layout(title=f'MAE vs MFte for MFtr = {selected_mftr}',
                          xaxis_title='MFte',
                          yaxis_title='MAE')

        clear_output(wait=True)
        display(fig)
        display(mftr_dropdown)

    mftr_dropdown.observe(update_plot, names='value')
    display(mftr_dropdown)
    update_plot(None)

In [ ]:
line_plot(grouped_rafael)

Dropdown(description='MFtr:', index=9, options=(np.float64(0.05), np.float64(0.15), np.float64(0.25), np.float…

In [5]:
line_plot(grouped_luiz)

Dropdown(description='MFtr:', index=3, options=(0.05, 0.25, 0.35, 0.45, 0.55, 0.65, 0.95, '0.05', '0.15', '0.2…

In [6]:
line_plot(grouped)

Dropdown(description='MFtr:', index=6, options=(0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, '0…